<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Descargando datos


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/integracion/python/descarga_datos.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

# 1. Descargando datos a la fuerza bruta

Vamos a ver primero como ir descargando datos y luego como lidiar con diferentes formatos. Es muy importante que, si los datos los vamos a cargar por única vez, descargar el conjunto de datos, tal como se encuentran, esto es `raw data`.

Vamos primero cargando las bibliotecas necesarias:

In [1]:

import os  # Para manejo de archivos y directorios
import urllib.request # Una forma estandard de descargar datos
# import requests # Otra forma no de las librerías de uso comun

import datetime # Fecha de descarga
import pandas as pd # Solo para ver el archivo descargado
import zipfile # Descompresión de archivos

Es importante saber en donde nos encontramos y crear los subdirectorios necesarios para guardar los datos de manera ordenada. Tambien es importante evitar cargar datos que ya han sido descargados anteriormente.

In [3]:
# pwd
print(os.getcwd())

#  AEstos son los datos que vamos a descargar y donde vamos a guardarlos
covid_url = "http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip"
covid_archivo = "./data/datos_abiertos_covid19.zip"
covid_dic_conceptos_url = "http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip"
covid_dic_conceptos_archivo = "./data/diccionario.zip"


/Users/juliowaissman/Documents/cursos/ic-mcd/ing-caract/ejemplos/integracion/python


In [ ]:
if not os.path.exists(covid_archivo):
    os.makedirs('./data')
    urllib.request.urlretrieve(covid_url, covid_archivo)  
    with zipfile.ZipFile(covid_archivo, "r") as zip_ref:
        zip_ref.extractall("./data/")
    
    urllib.request.urlretrieve(covid_dic_conceptos_url, covid_dic_conceptos_archivo)  
    with zipfile.ZipFile(covid_dic_conceptos_archivo, "r") as zip_ref:
        zip_ref.extractall("./data/")

    with open("../data/info.txt", 'w') as f:
        f.write("Archivos Covid Mexico\n")
        f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write("Desde: " + covid_url + "\n")
        f.write("Nombre: " + covid_archivo + "\n")
        f.write("Diccionario de datos desde: " + covid_dic_conceptos_url + "\n")
        f.write("Nombre: " + covid_dic_conceptos_archivo + "\n")
